# Convert CSV to Vaex HDF5 format(column based HDF5)

**Hardware:** n1-highmem-32, 32 vCPUs, 208 GB memory, 1TB SSD

In [1]:
import gc
import os

import dask.bag as db
import pandas as pd
import vaex
from dask.diagnostics import ProgressBar

In [2]:
CSV_DIR = '/data/csv'
VAEX_HDF5_DIR = '/data/vaex-hdf5'

In [3]:
def csv_to_vaex_hdf5(csv_file: str, hdf5_file: str)->None:
    assert csv_file.endswith('.csv.zip')
    assert hdf5_file.endswith('.hdf5')
    df = vaex.read_csv(csv_file)
    df.export_hdf5(hdf5_file, progress=False)
    del df
    gc.collect()

In [4]:
csv_to_vaex_hdf5('/data/csv/OKEx.Swap.XMR_USD.csv.zip', '/data/vaex-hdf5/OKEx.Swap.XMR_USD.hdf5')

In [5]:
def convert_dir(csv_dir: str, hdf5_dir: str)->None:
    files = os.listdir(csv_dir)
    with ProgressBar():
        db.from_sequence(files).map(
            lambda file: csv_to_vaex_hdf5(os.path.join(csv_dir, file), 
                                     os.path.join(hdf5_dir, file.replace('.csv.zip', '.hdf5')))
        ).compute()

In [6]:
convert_dir(CSV_DIR, VAEX_HDF5_DIR)

[########################################] | 100% Completed |  1min 28.6s
